In [1]:
!pip install catboost

In [2]:
from catboost.datasets import titanic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
train_df, test_df = titanic()
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
train_df.fillna(-999, inplace=True)
test_df.fillna(-999, inplace=True)

In [6]:
X = train_df.drop('Survived', axis=1)
y = train_df.Survived

In [7]:
print(X.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [8]:
categorical_features_indices = np.where( (X.dtypes!=np.float) & (X.dtypes != np.int))[0]
categorical_features_indices

array([ 2,  3,  7,  9, 10])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25, random_state=42)
X_test = test_df

In [10]:
from catboost import CatBoostClassifier,CatBoostRegressor, Pool, cv
from sklearn.metrics import accuracy_score

In [11]:
model = CatBoostClassifier(
    loss_function ='Logloss',
    eval_metric = 'Accuracy',
    custom_metric='F1',
    random_seed=42,
    logging_level='Silent'
)

In [12]:
model.fit(
    X_train,y_train,
    cat_features = categorical_features_indices,
    eval_set=(X_validation,y_validation),
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [13]:
print('Правильность на обучающей выборке: {:.3f}'.format(model.score(X_train,y_train)))
print('Правильность на проверочной выборке: {:.3f}'.format(model.score(X_validation,y_validation)))

Правильность на обучающей выборке: 0.919
Правильность на проверочной выборке: 0.839


In [16]:
model = CatBoostClassifier(
    loss_function = 'Logloss',
    eval_metric = 'Accuracy',
    custom_metric ='F1',
    l2_leaf_reg = 30,
    random_seed = 42,
    logging_level='Verbose'
)

In [ ]:
model.fit(
    X_train,y_train,
    cat_features = categorical_features_indices,
    eval_set=(X_validation,y_validation),
    plot=False
)

In [18]:
print('Правильность на обучающей выборке: {:.3f}'.format(model.score(X_train,y_train)))
print('Правильность на проверочной выборке: {:.3f}'.format(model.score(X_validation,y_validation)))

Правильность на обучающей выборке: 0.894
Правильность на проверочной выборке: 0.830


In [20]:
predictions = model.predict(X_test)
proba  = model.predict_proba(X_test)
print(predictions[:5])
print(proba[:5])

[0 0 0 0 1]
[[0.85333966 0.14666034]
 [0.52901528 0.47098472]
 [0.81259813 0.18740187]
 [0.86802974 0.13197026]
 [0.36647229 0.63352771]]


In [21]:
params ={
    'loss_function':'Logloss',
    'eval_metric':'Accuracy',
    'custom_metric':'F1',
    'random_seed':42,
    'logging_level':'Silent'
}

In [22]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation,y_validation, cat_features=categorical_features_indices)

In [23]:
model =CatBoostClassifier(**params)
model.fit(train_pool,eval_set=validate_pool,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [24]:
print('Правильность на обучающей выборке: {:.3f}'.format(model.score(X_train,y_train)))
print('Правильность на проверочной выборке: {:.3f}'.format(model.score(X_validation,y_validation)))

Правильность на обучающей выборке: 0.919
Правильность на проверочной выборке: 0.839


In [25]:
model.save_model('catboost_model.dump')